## Relationship between Github and Git
Github is host for git repositories, `git` as a command line tool is independent from `Github`, we don't need `Github` to use `git` and vice versa

Folders are called `trees` and files are called `blobs` in `git` terminology. Git uses a directed acylic graph to model history, instead of a linear sequence of snapshots of the histories of the files and folders. 

Each snapshots contains not only the `trees` and `blobs` but also metadata about the author, commit messages etc.  

## Data structure of git
psuedocode
```
type blob = array<byte>
# type tree maps the tree name to trees and blobs below it
type tree = map<string, tree|blob> 

# note the commit doesn't actually store the full commit or tree, it is just a pointer to the commit or the tree
type commit = struct{
    parents : array<commit> # a snapshot can have multiple parents
    author : string
    message : string
    snapshot : top-level tree
}

# storing blob/tree/commit on disk in a datatype called object
type object = blob | tree | commit

type objects = map<string, object> 
# maps an id (from hash) to the object

def store(obj):
    id = sha1(obj) # this is a hash function SHA-1
    objects[id] = obj

def load(id):
    return objects[id]
    
# references give human-readable names (for example branch names) to the ids generated through SHA-1 hash 
type references = map<string, string>
```
On a high level, all git commands are manipulations of the object data or the reference data

`git cat-file p hash_string` is a very cool command that shows what data structure the `hash_string` refers to

Git has a staging area, that tells `git` what changes should be included in the next snapshot we take (`git commit` generates a snapshot)

`git log` allows us to visualize the (flattened) version history, `git log --all --graph --decorate` however gives a graph visualization of the history, supplying this command with an extra `--oneline` flag will give a compact representation of the version history

`git commit -a` commits all the changes that were made to files that were already tracked by git, it doesn't include new things

`git add :/` commits everything top down from the repository

`master` is the reference that generally points to the latest commit of the repository

`HEAD` is a special reference that refers to which snapshot (commit) we are currently looking at, we can change the location of the `HEAD` by checking out at different snapshots (`git checkout`)

`git checkout hash_string` checks us out at a previous commit, we also don't need to supply the full `hash_string`, giving `git checkout` the first few characters of the `hash_string` will be enough. Alternatively we can give the reference name (for example `master`). In conclusion, `git checkout` moves the `HEAD` pointer as well as the content of the working directory based on where the `HEAD` pointer is pointing. 

`git checkout file_name` allows us to throw away the changes that we have made to the file and reverts the file to how it was in the snapshot that `HEAD` points to.

`git diff hash_string` shows what has changed between the current state (note, the current state doesn't have to be the same as the latest snapshot!), if the hash_string is not supplied, it compares the current state with `HEAD`.

## Branches and merging
`git branch` prints the current branches

`git branch branch_name` creates a new branch (which is just a reference) that points to wherever `HEAD` is pointing 

`git checkout branch_name` will switch us out to a different branch 

`git checkout -b branch_name` creates the branch and switches us to the new branch as well

`git merge` merges the branches, `git merge abort` aborts the merging. If there is a merge conflict, we will go in the conflict file, fix the conflict, readd the file, and then run `git merge --continue`

## Git remotes
`git remote` will list all the remote repositories that `git` is aware of 

`git remote add <name> <url>` add a new remote, typically the `<name>` will be `origin` if there is one remote by convention, and `<url>` can be the `github` url or even a folder on the local computer (see demo in class)

`git push <remote> <local branch>:<remote branch>`, example `git push origin master:master` 

`git branch --set-upstream-to=origin/master` will set the correspondence between the current branch that's being checked out on the local machine (whereever `HEAD` is pointing to) and the `master` branch on the `origin` remote repository

`git fetch <remote>` fetches the updates that have been made to the `<remote>` repository, and to move the local `master` branch up to date with the remote repository, we do a `git merge`. Alternatively, we can also do `git pull` which is equivalent to `git fetch` + `git merge`  

## Some other features
If the git repo is gigantic and has a lot of previous commits, and it is taking a long time to `clone` the entire history, then we can do `git clone --shallow` to only get the latest snapshot of the remote repository

`git add -p` allows us to stage modifications interactively, so we can selectively choose what modifications we want to commit and what to discard

`git blame` can figure out who edited which part

`git stash` will temporarily allow us to revert back to the latest commit, while stashing away the latest modifications, `git stash pop` puts us back to the stashed copy

`git bisect` use case: can figure out at which point a unit test started failing 

We can also get shell integration with `git`, to show what branch we are in through the command line